# Scikit-learn Regression - Workflow 구성하기
작성자: 고정훈

Regression은 아래와 같은 흐름(flow)을 대략적으로 따른다고 Basic Flow Tutoria에서 설명드렸습니다. 

* Preprocessing
    1. Data Munging
    2. One-hot-encoding
    3. Missing Data Processing
    4. Scaling
    5. Feature Selection
    6. Dimensionality Reduction
    
    
* Training
    1. Train/Test Set Split 
    2. Algorithm Selection
    3. Hyper-parameter Selection
    4. Training 
    5. Validation 
    
Data Munging은 데이터별로 다 다르기 때문에 정형화된 프로세스를 적용하기 어렵지만 그 외의 것들은 어느정도 정형화된 Workflow를 구성할 수 있습니다. 

## Example

Scikit-learn에서 제공하는 Lasso Algorithm을 이용해 Workflow를 만드는 예제를 구성해보겠습니다.

In [1]:
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.datasets import load_boston
from sklearn.metrics import r2_score


x, y = load_boston(return_X_y=True)  # 데이터를 불러옵니다.
y = np.reshape(y, (-1, 1))
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=2)

print('Original x_train shape: ', x_train.shape)  # 원 데이터의 sample 개수와 feature 개수를 shape을 통해 출력합니다. 

pf = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)  # 2차항 feature들을 만들어주는 클래스의 instance를 만듭니다. 
x_poly = pf.fit_transform(x_train)  #  2차항과 교차항을 포함해 feature array를 만듭니다. 
print('Polynomial x shape: ', x_poly.shape)  # feature array가 어떻게 바뀌었는지 출력해봅니다. 

Original x_train shape:  (379, 13)
Polynomial x shape:  (379, 104)


Lasso는 선형 모델입니다. 일반 linear regression은 feature의 개수가 sample의 개수보다 많으면 제대로 풀리지 않습니다(무수히 많은 해가 나옵니다). 
Lasso는 모델 파라미터에 L1 penalty를 부여하여 feature 개수가 sample 수보다 많아도 stable한 결과를 내주는 선형 모델입니다. 
선형 모델이기 때문에 원래의 feature를 그대로 쓰면 비선형성을 담을 수 없겠죠. 2차항 혹은 3차항 등을 포함한다면 비선형성을 어느정도 담을 수 있을 것입니다. 

위 코드에서는 scikit-learn의 PolynomalFeatures를 이용하여 feature들을 2차항으로 만들었습니다. 여기에는 각 feature들의 2차항과 교차항이 모두 포함되어 있습니다.
원래 feature 개수 13, 각 feature의 2차항 개수 13, 교차항 개수 78개 포함해서 총 104개로 feature 개수가 늘었습니다.

In [2]:
y = np.reshape(y, (-1, 1))

x_scaler = StandardScaler()
y_scaler = StandardScaler()
x_scaled = x_scaler.fit_transform(x_poly)
y_scaled = y_scaler.fit_transform(y_train)

regressor = LassoCV(max_iter=20000, cv=KFold(shuffle=True), random_state=1)  # LassoCV를 정의합니다. 
val_scores = cross_val_score(regressor, x_scaled, y_scaled.ravel(), cv=KFold(n_splits=5, shuffle=True))  # score를 구합니다. 
print('Validation Score: ', np.mean(val_scores))

regressor.fit(x_scaled, y_scaled.ravel())

Validation Score:  0.807959167228


LassoCV(alphas=None, copy_X=True,
    cv=KFold(n_splits=3, random_state=None, shuffle=True), eps=0.001,
    fit_intercept=True, max_iter=20000, n_alphas=100, n_jobs=1,
    normalize=False, positive=False, precompute='auto', random_state=1,
    selection='cyclic', tol=0.0001, verbose=False)

Lasso는 L1 penalty를 어느 강도로 줄지 결정하는 alpha 값을 정하는 것이 중요합니다. 이 alpha 값이 너무 작으면 over-fitting이 발생하고, 너무 크면 under-fitting이 발생합니다. 
LassoCV는 내부적으로 cross validation을 돌려 후보 중 최적의 alpha를 구하는 scikie-learn 클래스입니다. 

cross_val_score는 LassoCV 내부의 cross validation과 무관하게 그 바깥으로 cross validation을 돌리겠죠.

이렇게 workflow를 쭉 적어서 만들 수 있습니다. 새로운 데이터가 왔을 때 예측은 아래와 같이 그 경로를 그대로 따릅니다. 

In [3]:
x_test = pf.transform(x_test)  # 먼저 polynomial feature를 만듭니다. 
x_test = x_scaler.transform(x_test)  # scaling 합니다. 
y_pred_scaled = regressor.predict(x_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
test_score = r2_score(y_test, y_pred)
print('Test Score: ', test_score)

Test Score:  0.844666529908


매번 위 flow를 반복하면 됩니다. 그런데 잘 보시면 method가 반복되죠. fit_transform이 학습할 때 반복되었다면 predict 할 때는 transform이 반복됩니다. 규칙적이기 때문에 하나의 함수나 클래스로 쉽게 묶을 수 있겠네요. 해당 기능을 해주는 것이 바로 `Pipeline`입니다.

## Pipeline

scikit-learn은 통일된 인터페이스로 설계되어 있습니다. 어떤 작업을 수행하든 `fit`은 데이터에 적합시키는 데 사용하고 preprocessing 등에서는 `transform`이라는 용어로 데이터를 변형시키죠. unlabeled 데이터가 들어와서 regression이나 classification을 진행할 때는 `predict`를 사용하며, 성능을 볼 때는 `score`를 사용합니다. 

method의 규칙성은 workflow를 손쉽게 구성할 수 있게 해줍니다. 그리고 그 일을 해주는 것이 `Pipeline`이라는 클래스입니다. `Pipeline`을 써서 앞의 workflow를 다시 구성해보겠습니다.

In [4]:
import numpy as np
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.datasets import load_boston
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score


x, y = load_boston(return_X_y=True)
y = np.reshape(y, (-1, 1))
x_train, x_test, y_train, y_test = train_test_split(x, y)

pf = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)  # 2차항 feature들을 만들어주는 클래스의 instance를 만듭니다. 

x_scaler = StandardScaler()
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train)

regressor = LassoCV(max_iter=20000, cv=KFold(shuffle=True), random_state=1)  # LassoCV를 정의합니다. 

steps = [('poly', pf), ('scaler', x_scaler), ('regressor', regressor)]  # 차례로 진행항 operator의 이름과 instance를 tuple 형태로 list에 넣습니다.
pipeline = Pipeline(steps=steps)
val_scores = cross_val_score(pipeline, x_train, y_train_scaled.ravel())
print('Validation Score: ', np.mean(val_scores))

pipeline.fit(x_train, y_train_scaled.ravel())
y_pred_scaled = pipeline.predict(x_test)
y_pred = y_scaler.inverse_transform(y_pred_scaled)
test_score = r2_score(y_test, y_pred)
print('Test Score: ', test_score)

Validation Score:  0.808595216446
Test Score:  0.865388993891


`Pipeline`에 들어가는 인자는 *steps*입니다. *stpes*는 tuple의 list입니다. list 안의 tuple은 operator의 이름과 instance로 구성되어 있습니다. `Pipeline`을 `fit`하거나 `predict`하면 list 내의 operator가 연속적으로 실행됩니다. 앞의 operator가 실행한 후 결과물을 다음 operator로 넘기죠. 

앞서 한줄 한줄 적었던 동작을 한번에 수행합니다. 마치 하나의 instance처럼 다룰 수 있는 거죠. 

### Pipeline 동작

Scikit-learn에서 작업을 수행하는 opeartor는 크게 두가지가 있습니다. **transformer**와 **estimator**가 그것입니다. transformer는 `transform`이라는 method를 가지면서 x 데이터를 변형 시키는 역할을 합니다. 주로 preprocessing 단계의 일들을 수행하죠. estimator는 `predict`라는 method를 가지면서 x가 입력되었을 때 y를 예측하는 operator입니다. regression이나 classification을 수행하는 operator들이 여기에 해당합니다. 

- `Pipeline`에서 estimator는 제일 마지막에만 올 수 있습니다. 
- `Pipeline`에서 transformer는 어디에나 올 수 있습니다. 
- `Pipeline` instance를 `fit` 하면 transformer에서는 `fit_transform`이 연속적으로 실행됩니다. 
- `Pipeline` instance를 `predict`하면 `transform`이 연속적으로 실행되다 마지막 operator가 estimator이면 `predict`가 실행됩니다. 
- `Pipeline` instance를 `fit_predict`하면 `fit_transform`이 연속적으로 실행되다 마지막 operator가 estimator이면 `fit_predict`가 실행됩니다.
- `Pipeline` instance에서 `trasnform`이나 `fit_trasnform`을 쓰려면 마지막 operator가 transformer여야 합니다. 이때는 연속적으로 `transform` 혹은 `fit_transform`이 실행됩니다. 

### Pipeline의 장점

- 간결하게 workflow를 구성할 수 있습니다. 반복적으로 operator를 일일이 실행시키지 않아도 됩니다. 
- 하나의 opeartor처럼 작동하기 때문에 operator가 input으로 들어가는 scikit-learn의 다른 함수나 클래스에 편하게 입력할 수 있습니다.
- 최적의 Workflow를 찾을 때 유리합니다. 머신러닝은 전처리(preprocessing) 방법과 학습 알고리즘 간의 궁합이 중요합니다. 일일이 시도해서 찾는 건 굉장히 지루한 일입니다. 다음 Tutorial에서 배울 Model Selection 방법과 합쳐졌을 때 Pipeline은 강력한 힘을 발휘합니다. 

머신러닝은 학습방법만큼이나 전처리가 중요합니다. 학습방법은 어느정도 표준화가 되어 있기 때문에, 어쩌면 전처리가 더 중요할 수도 있습니다. 전처리와 학습을 묶어주는 Pipeline을 자유자재로 사용하신다면 분명 훨씬 빠르고 간결하게 솔루션을 만드실 수 있을 것입니다. 